In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]

'/home/mori/.gcloud/agent-engine-test-462504-001b7ad7b0a9.json'

## Cloud SQL Proxy を通して接続する

- DB インスタンスはプライベート IP のみを持つ設定にしている


In [9]:
# !nohup cloud-sql-proxy --address 127.0.0.1 --private-ip agent-engine-test-462504:asia-northeast1:agent-engine-test --port 5432 &

In [10]:
project_id = "agent-engine-test-462504"
region = "asia-northeast1"
instance_name = "agent-engine-test"
database_name = "testdb"
db_user = "postgres"
db_password = "postgres"

In [11]:
import psycopg2


try:
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=db_user,
        password=db_password,
        database=database_name,
    )
    cur = conn.cursor()

    # テーブル作成（すでに存在する場合はスキップ）
    cur.execute("""
        CREATE TABLE IF NOT EXISTS mytable (
            id SERIAL PRIMARY KEY,
            name TEXT,
            age INTEGER
        );
    """)

    # サンプルデータ挿入
    cur.execute("INSERT INTO mytable (name, age) VALUES (%s, %s);", ("Alice", 30))
    cur.execute("INSERT INTO mytable (name, age) VALUES (%s, %s);", ("Bob", 25))
    cur.execute("INSERT INTO mytable (name, age) VALUES (%s, %s);", ("Charlie", 35))

    # コミットして変更を確定
    conn.commit()

    # データの取得と表示
    cur.execute("SELECT * FROM mytable;")
    rows = cur.fetchall()

    for row in rows:
        print(row)

    cur.close()
    conn.close()
except Exception as e:
    print(f"Connection error: {e}")

Connection error: connection to server at "127.0.0.1", port 5432 failed: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.



## Cloud SQL Python Connector を通して接続する

- サービスアカウントのロールに `Cloud SQL クライアント` を追加する


In [5]:
project_id = "agent-engine-test-461308"
region = "us-central1"
instance_name = "agent-engine-test"
database_name = "testdb"
db_user = "postgres"
db_password = "postgres"

In [10]:
from google.cloud.sql.connector import Connector

connector = Connector()


def getconn():
    conn = connector.connect(
        f"{project_id}:{region}:{instance_name}",
        "pg8000",
        user=db_user,
        password=db_password,
        db=database_name,
    )
    return conn


try:
    with getconn() as conn:
        cur = conn.cursor()
        cur.execute("SELECT * FROM mytable;")
        rows = cur.fetchall()
        for row in rows:
            print(row)
        cur.close()
except Exception as e:
    print(e)

[1, 'Alice', 30]
[2, 'Bob', 25]
[3, 'Charlie', 35]
